## Part A: Multilingual Intent Detection

In [100]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code = True)

In [101]:
# Labeled Training data
raw_data = [
    ("I want to book a room for two adults.", "book_room"),
    ("क्या मैं अपनी बुकिंग को अगले हफ्ते के लिए रद्द कर सकता हूँ?", "cancel_booking"),
    ("Do you have any special offers available?", "request_special_offer"),
    ("होटल के पास प्रसिद्ध आकर्षण क्या हैं?", "ask_about_local_attractions"),
    ("मेरे कमरे की सफाई के बारे में मुझे शिकायत है।", "resolve_complaint"),
    ("Is there availability for a room on July 15th?", "check_availability"),
    ("Deluxe मेरे लिए रेट क्या है प्रति रात?", "inquire_room_rate"),
    ("मेरी बुकिंग को कैसे संशोधित करें?", "modify_booking"),
    ("I would like to extend my stay by two nights.", "extend_booking"),
    ("क्या मुझे लेट चेकआउट मिल सकता है?", "request_late_checkout"),
    ("Is WiFi available in the rooms?", "ask_wifi_availability"),
    ("मुझे होटल के रूम सर्विस मेन्यू की ज़रूरत है।", "request_room_service_menu"),
    ("Hi, are pets allowed in the hotel?", "ask_pet_policy"),
    ("क्या होटल में पार्किंग उपलब्ध है?", "ask_parking"),
    ("I'd like a wake-up call at 6 AM.", "request_wakeup_call"),
    ("Can I get an extra bed in my room?", "request_extra_bed"),
    ("Do you offer airport pickup services?", "ask_airport_pickup"),
    ("I'd like to speak to the manager.", "request_manager"),
    ("खिड़की के पास कमरा चाहिए।", "request_room_preference"),
    ("Please cancel my breakfast tomorrow.", "cancel_breakfast"),
    ("होटल में जिम और स्पा है?", "ask_gym_spa"),
    ("I need a quieter room, please.", "request_quiet_room"),
    ("Can I pre-order lunch for tomorrow?", "preorder_meal"),
    ("What’s the check-in policy?", "ask_checkin_policy"),
    ("Check-out timing kya hai?", "ask_checkout_policy"),
]

In [102]:
# Embeddings
train_texts = [x[0] for x in raw_data]
train_labels = [x[1] for x in raw_data]
train_embeddings = model.encode(train_texts)

In [103]:
 def predict_intent(text, top_k=1):
    query_embedding = model.encode([text])
    similarities = cosine_similarity(query_embedding, train_embeddings)[0]
    top_idx = np.argsort(similarities)[::-1][:top_k]
    return [(train_labels[i], similarities[i]) for i in top_idx]

In [104]:
test_sentences = [
    "मैं अगले सप्ताह के लिए अपनी बुकिंग रद्द करना चाहता हूँ।",
    "Is there a gym and spa in the hotel?",
    "Deluxe रूम का रेट क्या है?",
    "I want to extend my stay by two days.",
    "क्या मुझे सुबह 6 बजे के लिए वेक-अप कॉल मिल सकता है?",
    "I'd like to book a room with a sea view.",
    "Do you provide pickup from the airport?",
    "मेरे रूम में वाईफाई है क्या?",
    "I want to cancel my breakfast tomorrow.",
    "Can I request a quieter room?"
]

In [105]:
# Inference
for sentence in test_sentences:
    predictions = predict_intent(sentence)
    top_intent, score = predictions[0]
    print(f"Query: {sentence}\nPredicted Intent: {top_intent} (Score: {score:.2f})\n")


Query: मैं अगले सप्ताह के लिए अपनी बुकिंग रद्द करना चाहता हूँ।
Predicted Intent: cancel_booking (Score: 0.91)

Query: Is there a gym and spa in the hotel?
Predicted Intent: ask_pet_policy (Score: 0.68)

Query: Deluxe रूम का रेट क्या है?
Predicted Intent: inquire_room_rate (Score: 0.92)

Query: I want to extend my stay by two days.
Predicted Intent: extend_booking (Score: 0.90)

Query: क्या मुझे सुबह 6 बजे के लिए वेक-अप कॉल मिल सकता है?
Predicted Intent: cancel_booking (Score: 0.81)

Query: I'd like to book a room with a sea view.
Predicted Intent: book_room (Score: 0.63)

Query: Do you provide pickup from the airport?
Predicted Intent: ask_airport_pickup (Score: 0.89)

Query: मेरे रूम में वाईफाई है क्या?
Predicted Intent: request_room_service_menu (Score: 0.76)

Query: I want to cancel my breakfast tomorrow.
Predicted Intent: cancel_breakfast (Score: 0.93)

Query: Can I request a quieter room?
Predicted Intent: request_quiet_room (Score: 0.92)



## Part B: Basic Conversation Analysis

In [110]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code = True)

In [111]:
conversation_history = [
    "Welcome to Hotel Booking Chatbot. How can I assist you today?",
    "मैं दो वयस्कों और एक बच्चे के लिए कमरा बुक करना चाहता हूँ.",
    "Sure, when would you like to check-in?",
    "कल शाम को, कृपया।",
    "Alright, let me check for availability.",
    "बहुत बढ़िया, धन्यवाद!",
    "You're welcome. Here are your booking details.",
    "नमस्ते! मैं एक सूट के लिए बुकिंग करना चाहता हूँ।",
    "कृपया कमरा बुक करें, जिसमें सभी सुविधाएँ शामिल हों।",
    "आपकी बुकिंग की जांच करने के लिए आपका धन्यवाद।",
    "कमरा उपलब्ध है। कृपया अपनी चेक-इन तारीख की पुष्टि करें।",
    "मुझे कल शाम को चेक-इन करना है।",
    "ठीक है, आपके बुकिंग की पुष्टि हो गई है।",
    "Thank you for booking with us!",
    "धन्यवाद! अगली बार आपका स्वागत है।",
    "Welcome to our hotel booking service. How may I assist you?",
    "I'd like to inquire about room availability for two adults and one child.",
    "Sure, what dates are you interested in?",
    "I'm looking to check-in tomorrow evening.",
    "ठीक है, मुझे उपलब्धता की जांच करने दें।",
    "धन्यवाद, कृपया मुझे जानकारी दें।",
    "आपके लिए एक सूट उपलब्ध है। कृपया अपनी चेक-इन तारीख की पुष्टि करें।",
    "Let me check for availability for you.",
    "Thank you, that would be great!",
    "You're welcome. Here are the available rooms for your requested dates.",
    "Thank you! I'll check them out.",
    "नमस्ते! मैं एक सूट बुक करना चाहता हूँ।",
    "आपको किस तारीख को चेक-इन करना है?",
    "मुझे कल शाम को आना है।",
    "बहुत अच्छा, धन्यवाद।",
    "आपकी बुकिंग की पुष्टि के लिए धन्यवाद।",
    "Thank you for choosing our hotel.",
    "मुझे आपका आदान-प्रदान करने में खुशी हुई।",
    "Welcome to our hotel booking service. How may I assist you?",
]

In [112]:
embeddings = model.encode(conversation_history)

conversation_pairs = []
for i in range(0, len(conversation_history) - 1, 2):
    user = conversation_history[i]
    bot = conversation_history[i + 1]
    conversation_pairs.append((user, bot))

user_inputs = [pair[0] for pair in conversation_pairs]
user_embs = model.encode(user_inputs)

In [113]:
def generate_response(user_query, top_k=1):
    query_emb = model.encode([user_query])
    sim_scores = cosine_similarity(query_emb, user_embs)[0]

    top_idx = np.argsort(sim_scores)[-top_k:][::-1]

    print(f"\nUser: {user_query}")
    for idx in top_idx:
        print(f"Chatbot{conversation_pairs[idx][1]}")

In [114]:
test_queries = [
    "Can I book a room for three people?",
    "मुझे कल के लिए एक रूम चाहिए।",
    "Do you have suites with all amenities?",
    "क्या मैं अपनी बुकिंग की तारीख बदल सकता हूँ?",
    "Can I get a late check-out?",
    "What are your check-in times?"
]

In [115]:
for q in test_queries:
    generate_response(q)



User: Can I book a room for three people?
ChatbotSure, what dates are you interested in?

User: मुझे कल के लिए एक रूम चाहिए।
Chatbotबहुत अच्छा, धन्यवाद।

User: Do you have suites with all amenities?
ChatbotThank you! I'll check them out.

User: क्या मैं अपनी बुकिंग की तारीख बदल सकता हूँ?
Chatbotमुझे कल शाम को चेक-इन करना है।

User: Can I get a late check-out?
Chatbotकल शाम को, कृपया।

User: What are your check-in times?
Chatbotकल शाम को, कृपया।


This model has been lightly fine-tuned. We can get better results by giving it more data, which is another problem statement entirely. Due to the limited time, I'm limiting the training to smaller datasets.

One more thing to note is that in the case of Hotel Management System based chatbot, we only need to have a limited number of responses, like the zomato chatbot.

Using a lightweight LLM does not do very well in getting proper responses.This is because the amount of context that they can capture is very limited.While it did good on the classification task, it does not provide very good answers in analysing deeper patterns. This experiment can be taken further to include fine-tuning or trying out with a heavier model.